# Setup of Noteboook

The follwing code clones the github repository with course files. 
Subsequently it imports all libraries and custom modules needed for this notebook

In [ ]:
!git clone https://github.com/DataHow/analytics-course-scripts.git

In [10]:
# import libaries
import pandas as pd
import numpy as np
import scipy
import copy
import importlib  
import scipy.integrate
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import GridSearchCV, validation_curve
from sklearn.metrics import mean_squared_error

# import custom modules
simulator = importlib.import_module("analytics-course-scripts.scripts.modules.simulator")
modelhelpers = importlib.import_module("analytics-course-scripts.scripts.modules.modelhelpers")

# Import or Generate Dataset

In the beginning of each notebook you can make a decision whether you run the notebooks with already pre-generated runs which will be just imported or you will generate your own doe dataset from insilico model. For further information about the process emulator for cell culture fed-batch processes, check the script "00_Process_Characterization.ipynb" or the Powerpoint presentation "Simplified InSilico Model.pptx".

* For dataset import set `select_dataset_source = "import"`.
* For generating data set `select_dataset_source = "generate"`.

For first time running the notebook, we recommend choosing the "import" option.

 

In [3]:
""" Select dataset source, one of (import, generate) """
DATASET_SOURCE = "import"

In [4]:
if DATASET_SOURCE == "import":
    # Define filename to import
    FILENAME = "owu.csv"
    FILEPATH = "/content/analytics-course-scripts/scripts/datasets/"
    # Import OWU data
    doe = pd.read_csv(FILEPATH+FILENAME.replace(".csv","_doe.csv"),index_col=None, usecols =["feed_start","feed_end","Glc_feed_rate","Glc_0","VCD_0"])
    owu = pd.read_csv(FILEPATH+FILENAME,index_col=None, usecols = ["X:VCD", "X:Glc", "X:Lac", "X:Titer","W:Feed"])
    owu.index = pd.MultiIndex.from_product([list(range(int(len(owu)/15))),list(range(15))], names=["run","time"])
    # Import OWU Test data
    FILENAME = "owu_test.csv"
    doe_test = pd.read_csv(FILEPATH+FILENAME.replace(".csv","_doe.csv"),index_col=None, usecols =["feed_start","feed_end","Glc_feed_rate","Glc_0","VCD_0"])
    owu_test = pd.read_csv(FILEPATH+FILENAME,index_col=None, usecols = ["X:VCD", "X:Glc", "X:Lac", "X:Titer","W:Feed"])
    owu_test.index = pd.MultiIndex.from_product([list(range(int(len(owu)/15))),list(range(15))], names=["run","time"])

In [5]:
if DATASET_SOURCE == "generate":
    # DOE Dataset definition (variable = [lower bound, upper bound]) 
    # Model parameters: Dictate the behaviour of the cell process
    MU_G_MAX = 0.05;
    MU_D_MAX = 0.025;
    K_G_GLC  = 1;
    K_I_LAC  = 30;
    K_D_LAC  = 50;
    K_GLC    = 0.04;
    K_LAC    = 0.06;
    K_PROD   = 1;
    # Process parameters: Conditions at which process is run
    FEED_START = [1, 4]
    FEED_END = [8, 12]
    GLC_FEED_RATE = [5, 20]
    GLC_0 = [10, 80.0]
    VCD_0 = [0.1, 1.0]
    # Number of experiments to generate
    NUM_RUNS = 40
    # Filename and filepath for the dataset 
    FILENAME = "owu.csv"
    FILEPATH = "/content/"
    # Collect parameters to dictionary
    VAR_LIMS = {"mu_g_max":MU_G_MAX, "mu_d_max": MU_D_MAX, "K_g_Glc" : K_G_GLC, "K_I_Lac" : K_I_LAC, "K_d_Lac" : K_D_LAC, "k_Glc" : K_GLC, "k_Lac" : K_LAC, "k_Prod" : K_PROD,
        "feed_start" : FEED_START,"feed_end" : FEED_END, "Glc_feed_rate" : GLC_FEED_RATE, "Glc_0" : GLC_0, "VCD_0" : VCD_0}
    # Generate Dataset
    data = simulator.generate_data(VAR_LIMS, NUM_RUNS, FILENAME)
    # Import DOE
    doe = pd.read_csv(FILEPATH+FILENAME.replace(".csv","_doe.csv"),index_col=None, usecols =["feed_start","feed_end","Glc_feed_rate","Glc_0","VCD_0"])
    # Import OWU
    owu = pd.read_csv(FILEPATH+FILENAME,index_col=None, usecols = ["X:VCD", "X:Glc", "X:Lac", "X:Titer","W:Feed"])
    owu.index = pd.MultiIndex.from_product([list(range(NUM_RUNS)),list(range(15))], names=["run","time"])

# Regression Modelling

In this section, we first load / import the train and test experiments that were created in the Process_Characterization script. Alternatively you can generate your own training data using generate option. 

These experimental data will be used to create models with the aim of predicting the final titer. 

Several different types of models will be considered.
* *Initial Conditions Model*: takes only the process condition and initial conditions as inputs. Black box type model, which disregards the process behaviour and consideres only outcome.
* *Historical Model*: takes full process history into account as inputs. This model however needs to be able to handle high-dimensional & colinear data.

# Initial Conditions Models

From OWU train and test dataset we create BWU data, DOE data and Target data.
DOE data contains the inditial parameter conditions (or Z variable) for the experiments.
Target data contain just the final values of Titer at day 14 (or Aggregates).

In [11]:
# Train set
bwu = simulator.generate_bwu(owu)
tar = simulator.generate_y(bwu,return_aggr=False)
# Test set
bwu_test = simulator.generate_bwu(owu_test)
tar_test = simulator.generate_y(bwu_test,return_aggr=False)

In [7]:
# Visualize input data and output data
dataset = copy.copy(doe) # inputs
dataset["Target"] = tar # outputs
dataset

,feed_start,feed_end,Glc_feed_rate,Glc_0,VCD_0,Target
0,2.500000,10.000000,12.500000,45.000000,0.550000,1565.310032
1,1.948980,10.244898,18.622449,40.714286,0.274490,854.449233
2,3.295918,9.346939,14.642857,17.857143,0.127551,1206.096655
3,3.479592,9.020408,15.561224,47.857143,0.843878,1587.937536
4,1.459184,9.591837,7.295918,33.571429,0.421429,811.55299
5,3.418367,9.265306,8.214286,45.000000,0.403061,841.498708
6,1.153061,8.448980,17.091837,67.857143,0.862245,645.86524
7,2.438776,8.122449,14.336735,10.714286,0.898980,978.32119
8,2.744898,11.551020,9.132653,27.857143,0.292857,1026.259363
9,2.316327,9.183673,5.459184,75.000000,0.256122,1307.342791


## Multiple Linear Regression 

We first train/fit a Linear Regression model to the final titer, using the data loaded above.

We then characterize the performance of the model using ANOVA, where the user can have information about the importance of each coefficient and of the overal model.

### Create linear model with preprocessing

Here we create a linear model to predict the value of the final titer given the initial conditions.

* Input matrix: "doe". This corresponds to the values of the manipulated process parameters for each experiment.
* Output target: "tar". This corresponds to the final value of titer at the end of each experiment. (or aggregates)

In [8]:
doe.isna()

,feed_start,feed_end,Glc_feed_rate,Glc_0,VCD_0,Target
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
5,False,False,False,False,False,False
6,False,False,False,False,False,False
7,False,False,False,False,False,False
8,False,False,False,False,False,False
9,False,False,False,False,False,False


In [9]:
# Fit Linear Model
datascaler = StandardScaler()
does = datascaler.fit_transform(doe.values)

X = sm.add_constant(does)
y = tar.values
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

TypeError: ignored

### Evaluate model prediction on train set and model inference

In this section, we fit a different linear model that is more flexible regarding the inputs. We mainly check the predictions of the models versus the training data for the final titer.


In [ ]:
# Use data normalization
use_norm = True
# Use bias term
use_bias = True
# Use polynomial features of degree
use_degree = 1
# Use only interaction terms form higher degree polynomial features
use_inter = False

In [ ]:
# Define pipeline
pfeatures = PolynomialFeatures(degree=use_degree,interaction_only=use_inter, include_bias=use_bias)
pscaler = StandardScaler(with_mean=use_norm,with_std=use_norm)
lm = LinearRegression()
pipe = Pipeline([('features', pfeatures),('scaler', pscaler), ('model', lm)])
# Fit model
X = doe
y = tar
pipe.fit(X,y)

Pipeline(steps=[('features', PolynomialFeatures(degree=1)),
                ('scaler', StandardScaler()), ('model', LinearRegression())])

In [ ]:
# Model coefficients
columns = pfeatures.get_feature_names_out()
coefficients = lm.coef_
fig = px.bar(x=list(columns),y=coefficients.reshape(-1),title="Linear model coefficients",labels={'x':"Variables", 'y':"Estimated Coefficients"})
fig.show()

### Evaluate model prediction on train and test set

In [ ]:
# Plot predictions in train set
yhat = pipe.predict(X)
fig = px.scatter(x=y.values.reshape(-1),y=yhat.reshape(-1), title="Observed vs Predicted on Train Set; R^2 = "+str(round(pipe.score(X,y),3)), labels=dict(x="Observed Titer", y="Predicted Titer"))
fig.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1, 'layer': 'below'}])
fig.show()
# Plot predictions in test set
X_test = doe_test
y_test = tar_test
yhat_test = pipe.predict(X_test)
fig = px.scatter(x=y_test.values.reshape(-1),y=yhat_test.reshape(-1), title="Observed vs Predicted on Test Set; R^2 = "+str(round(pipe.score(X_test,y_test),3)), labels=dict(x="Observed Titer", y="Predicted Titer"))
fig.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1, 'layer': 'below'}])
fig.show()

In [ ]:
print("Error in test set prediction")
# Absolute RMSEP
rmse_abs = mean_squared_error(y_test, yhat_test,squared=False)
print('Absolute RMSEP: ',rmse_abs)
# Relative RMSEP
rmse_rel = mean_squared_error(y_test, yhat_test,squared=False) / np.std(np.array(y_test))
print('Relative RMSEP: ',rmse_rel)

Error in test set prediction
Absolute RMSEP:  339.09453647105425
Relative RMSEP:  1.1729649813946397


## PLS Model

In this section, we keep the same data structure but we train a PLS model.

### Create a PLS model with specific latent variables

Create a PLS model to predict the value of the final titer given the initial conditions.

* Input matrix: "doe". This corresponds to the values of the manipulated process parameters for each experiment.
* Output target: "tar". This corresponds to the final value of titer at the end of each experiment. (or aggregates)

Select the number of latent variables for the model (the maximum number of latent variables is 5, equal to the number of variables in the input matrix).





In [ ]:
# Use data normalization
use_norm = True
# Number of latent variables
N_LV = 5

In [ ]:
# Define Pipeline
pscaler = StandardScaler(with_mean=use_norm,with_std=use_norm)
pls = PLSRegression(n_components=N_LV)
pipe = Pipeline([('scaler', pscaler), ('model', pls)])
# Train PLS model
X = doe
y = tar
pipe.fit(X,y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model', PLSRegression(n_components=5))])

### Evaluate model on train and test set


In [ ]:
# Plot predictions in train set
yhat = pipe.predict(X)
fig = px.scatter(x=y.values.reshape(-1),y=yhat.reshape(-1), title="Observed vs Predicted on Train Set; R^2 = "+str(round(pipe.score(X,y),3)), labels=dict(x="Observed Titer", y="Predicted Titer"))
fig.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1, 'layer': 'below'}])
fig.show()
# Plot predictions in test set
X_test = doe_test
y_test = tar_test
yhat_test = pipe.predict(X_test)
fig = px.scatter(x=y_test.values.reshape(-1),y=yhat_test.reshape(-1), title="Observed vs Predicted on Test Set; R^2 = "+str(round(pipe.score(X_test,y_test),3)), labels=dict(x="Observed Titer", y="Predicted Titer"))
fig.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1, 'layer': 'below'}])
fig.show()

In [ ]:
# Absolute RMSEP
rmse_abs = mean_squared_error(y_test, yhat_test,squared=False)
print('Absolute RMSEP: ',rmse_abs)
# Relative RMSEP
rmse_rel = mean_squared_error(y_test, yhat_test,squared=False) / np.std(np.array(y_test))
print('Relative RMSEP: ',rmse_rel)

Absolute RMSEP:  339.0945364710543
Relative RMSEP:  1.17296498139464


### Plot variance explained in the Y

Similarly as in PCA script, we can look at explained variance by the principal components. However the difference being that in PCA we are trying to maximize the variance explained. In PLS we are trying to maximize the covarinace with the target.

In [ ]:
# Plot Explained Variance plots
scores = pls.x_scores_
expl_var = np.var(scores, axis=0)
expl_var_ratio = expl_var / np.sum(expl_var)
fig = px.line(x=list(range(N_LV)), y=np.cumsum(expl_var_ratio), color=px.Constant("Cumulative explained variance"), labels=dict(x="Principal component index", y="Explained Variance Ratio", color="Legend"))
fig.add_bar(x=list(range(N_LV)), y=expl_var_ratio, name="Individual explained variance")
fig.show()


In [ ]:
# Principal component on x-axis
select_x_pca = 1
# Principal component on y-axis
select_y_pca = 2

In [ ]:
# Plot loadings
loadings = pls.x_loadings_
fig = px.scatter(x=pls.x_scores_[:,select_x_pca-1],y=pls.x_scores_[:,select_y_pca-1], title="PLS Score and loadings plot",labels={'x':"PC - "+ str(select_x_pca) , 'y':"PC -"+ str(select_y_pca)})
for i, feature in enumerate(doe.columns):
    fig.add_shape(type='line', x0=0, y0=0, x1=loadings[i,select_x_pca-1], y1=loadings[i,select_y_pca-1])
    fig.add_annotation(x=loadings[i,select_x_pca-1], y=loadings[i,select_y_pca-1], ax=0, ay=0, xanchor="center", yanchor="bottom", text=feature)
fig.show()

### Plot VIP scores

In [ ]:
# Model coefficients
columns = X.columns
coefficients = modelhelpers.vip(X,y,pls)
fig = px.bar(x=list(columns),y=coefficients.reshape(-1),title="Variable Importance in PLS", labels={'x':"Variables", 'y':"VIP values"})
fig.show()


# Historical Model

In an so-called historical model, the data from different experiments are ordered into a batch-wise unfolded (BWU) matrix (i.e., every row corresponds to an experiment).

The BWU can be used to compute final properties of the experiment, like CQAs, which are typically the effect of the cumulated effect of the experiment profile.

In this example, we will use the BWU matrix to predict the final value of titer. Clearly, titer information are removed from the BWU matrix.



## Multiple Linear Model

Unlinke in the initial conditions model, there are several problems in using an linear regression / response surface model for modelling historical data. The main problem is that MLR requires our dataset $X$ (the BWU data) to have more rows (samples) than columns (variables). Second is that if $X^T X$ has full rank due only to noise, the inverse is unstable and small changes in noise realization can produce dramatically different results.

If we look at the shape of our BWU matrix, that it already has 75 variables and only 50 samples which makes it dificult to fit linear model.

In [ ]:
bwu.shape

(50, 75)

## PLS Model

We demonstrate that with more information and more variables, the PLS becomes a model with good predictions and low RMSEP.

### Create BWU matrix

Here the BWU matrix is created. The vaues of the manipulated variables are added as columns at the beginning of the matrix. The following pre-processing was performed.

* Remove titer (and also lactate)
* Remove exceeding days
* Eliminate invariant columns
* Remove linearly dependent columns
* Add process parameters at the beginning
* Create a PLS model from the initial design to the final titer


In [ ]:
# Define number of days to include
N_days = 13

In [ ]:
X = bwu
remove_columns = []
# Remove X:Titer 
remove_columns.extend([c for c in bwu.columns if c.startswith('X:Titer')])
# Remove X:Lactate
remove_columns.extend([c for c in bwu.columns if c.startswith('X:Lac')])
# Remove W:Feed
remove_columns.extend([c for c in bwu.columns if c.startswith('W:Feed')])
# Remove Initial conditions
remove_columns.extend([c for c in bwu.columns if c.endswith(':0')])
# Remove exceeding days
for d in range(N_days,15):
    remove_columns.extend([c for c in bwu.columns if c.endswith(":"+str(d))])
# Remove invariant 
remove_columns.extend(list(bwu.columns[~(bwu != bwu.iloc[0]).any().values]))

# Add and remove columns
X = X.drop(set(remove_columns),axis=1)
X = pd.concat([doe, X], axis=1)

X.columns

Index(['feed_start', 'feed_end', 'Glc_feed_rate', 'Glc_0', 'VCD_0', 'X:VCD:1',
       'X:VCD:2', 'X:VCD:3', 'X:VCD:4', 'X:VCD:5', 'X:VCD:6', 'X:VCD:7',
       'X:VCD:8', 'X:VCD:9', 'X:VCD:10', 'X:VCD:11', 'X:VCD:12', 'X:Glc:1',
       'X:Glc:2', 'X:Glc:3', 'X:Glc:4', 'X:Glc:5', 'X:Glc:6', 'X:Glc:7',
       'X:Glc:8', 'X:Glc:9', 'X:Glc:10', 'X:Glc:11', 'X:Glc:12'],
      dtype='object')

### Train Historical PLS model

In [ ]:
# Define number of latent variables
N_LV = 5
# Use data normalization
use_norm = True

In [ ]:
# Define Pipeline
pscaler = StandardScaler(with_mean=use_norm,with_std=use_norm)
pls = PLSRegression(n_components=N_LV)
pipe = Pipeline([('scaler', pscaler), ('model', pls)])
# Train PLS model
y = tar
pipe.fit(X,y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model', PLSRegression(n_components=5))])

### Evaluate model on train and test set




In [ ]:
# Plot predictions in train set
yhat = pipe.predict(X)
fig = px.scatter(x=y.values.reshape(-1),y=yhat.reshape(-1), title="Observed vs Predicted on Train Set; R^2 = "+str(round(pipe.score(X,y),3)), labels=dict(x="Observed Titer", y="Predicted Titer"))
fig.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1, 'layer': 'below'}])
fig.show()
# Plot predictions in test set
X_test = bwu_test
y_test = tar_test
X_test = X_test.drop(set(remove_columns),axis=1)
X_test = pd.concat([doe_test, X_test], axis=1)
yhat_test = pipe.predict(X_test)
fig = px.scatter(x=y_test.values.reshape(-1),y=yhat_test.reshape(-1), title="Observed vs Predicted on Test Set; R^2 = "+str(round(pipe.score(X_test,y_test),3)), labels=dict(x="Observed Titer", y="Predicted Titer"))
fig.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1, 'layer': 'below'}])
fig.show()

In [ ]:
# Absolute RMSEP
rmse_abs = mean_squared_error(y_test, yhat_test,squared=False)
print('Absolute RMSEP: ',rmse_abs)
# Relative RMSEP
rmse_rel = mean_squared_error(y_test, yhat_test,squared=False) / np.std(np.array(y_test))
print('Relative RMSEP: ',rmse_rel)

Absolute RMSEP:  44.912690726879084
Relative RMSEP:  0.1553578951494958


In [ ]:
# Principal component on x-axis
select_x_pca = 1
# Principal component on y-axis
select_y_pca = 2

In [ ]:
# Plot loadings
loadings = pls.x_loadings_
#fig = px.scatter(x=pls.x_scores_[:,select_x_pca-1],y=pls.x_scores_[:,select_y_pca-1])
fig = px.scatter(x=[0,0], y=[0,0], title = "Loadings of PLS model", labels={'x':"PC - "+ str(select_x_pca) , 'y':"PC -"+ str(select_y_pca)})
for i, feature in enumerate(X.columns):
    fig.add_shape(type='line', x0=0, y0=0, x1=loadings[i,select_x_pca-1], y1=loadings[i,select_y_pca-1])
    fig.add_annotation(x=loadings[i,select_x_pca-1], y=loadings[i,select_y_pca-1], ax=0, ay=0, xanchor="center", yanchor="bottom", text=feature)
fig.show()

In [ ]:
# Model coefficients
columns = X.columns
coefficients = modelhelpers.vip(X,y,pls)
fig = px.bar(x=list(columns),y=coefficients.reshape(-1),title="Variable Importance in PLS", labels={'x':"Variables", 'y':"VIP value"})
fig.show()

### Hyperparameter Optimization in Cross Validation


In this section, we will simulate a typical cross- validation to define the optimal number of latent variables.


Nfold PLS models are trained using (Nfolds-1) folds. For each model, the sum of squared residuals (SSR) is calculated and summed up..

This is repeated for different numbers of latent variables. The number of latent variables returning the least value of the RMSE is chosen as optimal.
A second criterion is selected, namely the adjusted R-squared (adj. R^2), which is weighting the effect of the number of latent variables, i.e., if two values of the number of latent variables are returning a similar value of the SSR, then the one using less variables is chosen to be more likely to produce robust predictions.:


In [ ]:
# Range of number of latent variables to optimize for.
range_LV = range(1, 26)
# Use k Fold for Cross Validation
use_folds = 10

In [ ]:
# Define Pipeline
pscaler = StandardScaler(with_mean=use_norm,with_std=use_norm)
pls = PLSRegression(n_components=N_LV)
pipe = Pipeline([('scaler', pscaler), ('pls', pls)])
# Train PLS model
pipe.fit(X,y)
# Obtain cross validation curve
train_eval, valid_eval = validation_curve(pipe, X, y, param_name = "pls__n_components", param_range=list(range_LV),scoring = 'neg_root_mean_squared_error')


In [ ]:
train_score=-np.mean(train_eval,axis=1)
valid_score=-np.mean(valid_eval,axis=1)
train_std = np.std(train_eval,axis=1)
valid_std = np.std(valid_eval,axis=1)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( x=list(range_LV), y=train_score, error_y=dict(type='data',array=train_std,visible=True), name="Training"))
fig.add_trace(go.Scatter( x=list(range_LV), y=valid_score, error_y=dict(type='data',array=valid_std,visible=True), name="Validation"))
fig.update_layout(title="Hyperparameter Optimization in PLS",xaxis_title="Number of Latent Variables",yaxis_title="RMSE",legend_title="Evaluation type")
fig.show()


# Task: 

1. What is the optimal number of latent variables to use for Historical PLS model
2. Check model diagnostics by plotting residuals and verify if the modelling assumptions are satisfied.
